### Dependencies and Setup

In [1]:
import pandas as pd
import csv
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import folium 
import matplotlib

### EXTRACT

In [2]:
# File to Load
crime_to_load = "Resources/Baltimore_2019_Crime_Only.csv"
cctv_to_load = "Resources/CCTV_hoods.csv"

In [3]:
# Read the CSV files
crime_df = pd.read_csv(crime_to_load)
cctv_df = pd.read_csv(cctv_to_load)

In [4]:
crime_df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,District,Neighborhood,Total Incidents
0,07/06/2019,01:30:00,5B,0 HILLSIDE RD,BURGLARY,NaN,NORTHERN,ROLAND PARK,1
1,07/06/2019,09:00:00,7A,1600 APPLETON ST,AUTO THEFT,NaN,WESTERN,EASTERWOOD,1
2,07/06/2019,07:03:00,6E,2500 EDGECOMBE CIR N,LARCENY,NaN,NORTHERN,PARKLANE,1
3,07/06/2019,02:30:00,3AK,PL & N HIGHLAND AV,ROBBERY - STREET,KNIFE,SOUTHEAST,NaN,1
4,07/06/2019,10:05:00,6C,700 WASHINGTON BLVD,LARCENY,NaN,SOUTHERN,WASHINGTON VILLAGE,1


In [5]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude,neighborhood
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,NorthEutawStreet
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,RidgleysDelight
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,WestBaltimoreStreet
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,RidgleysDelight
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,RidgleysDelight


In [6]:
#sample crime table to visualize crime
sample = crime_df.sample(5)
sample

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,District,Neighborhood,Total Incidents
5266,05/26/2019,20:00:00,7A,3100 GOUGH ST,AUTO THEFT,NaN,SOUTHEAST,HUDSON-HIGHLANDTOWN,1
6004,05/20/2019,17:00:00,5D,3000 FLEETWOOD AVE,BURGLARY,NaN,NORTHEAST,WESTFIELD,1
119,07/05/2019,21:30:00,6E,5800 EDGEPARK RD,LARCENY,NaN,NORTHEAST,LOCH RAVEN,1
21724,01/05/2019,14:00:00,6E,1200 N BROADWAY,LARCENY,NaN,EASTERN,OLIVER,1
17966,02/06/2019,17:30:00,6D,4000 GLEN AVE,LARCENY FROM AUTO,NaN,NORTHWEST,GLEN,1


### Create database connection


In [7]:
connection_string = "postgres:postgres@localhost:5432/postgres"
engine = create_engine(f'postgresql://{connection_string}')

### Confirm tables

In [8]:
engine.table_names()

['crime', 'cctv']

In [9]:
crime_df.to_sql(name='crime', con=engine, if_exists='append', index=True)

In [10]:
cctv_df.to_sql(name='cctv', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying both sets ondependently

In [11]:
pd.read_sql_query('select * from crime', con=engine).head()

,index,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,District,Neighborhood,Total Incidents
0,0,07/06/2019,01:30:00,5B,0 HILLSIDE RD,BURGLARY,None,NORTHERN,ROLAND PARK,1
1,1,07/06/2019,09:00:00,7A,1600 APPLETON ST,AUTO THEFT,None,WESTERN,EASTERWOOD,1
2,2,07/06/2019,07:03:00,6E,2500 EDGECOMBE CIR N,LARCENY,None,NORTHERN,PARKLANE,1
3,3,07/06/2019,02:30:00,3AK,PL & N HIGHLAND AV,ROBBERY - STREET,KNIFE,SOUTHEAST,None,1
4,4,07/06/2019,10:05:00,6C,700 WASHINGTON BLVD,LARCENY,None,SOUTHERN,WASHINGTON VILLAGE,1


In [12]:
pd.read_sql_query('select * from cctv', con=engine).head()

,cameraNumber,cameraLocation,cameraProject,Location 1,latitude,longitude,neighborhood
0,1,Eutaw and Lexington Market,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,NorthEutawStreet
1,2,Eutaw and Fayette,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,RidgleysDelight
2,3,Eutaw and Baltimore,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,WestBaltimoreStreet
3,4,Eutaw and Redwood,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,RidgleysDelight
4,5,Eutaw and Lombard,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,RidgleysDelight


### TRANSFORM

In [13]:
# Changing "Location 1" form cctv_df to 'latitude' and 'longitiude'
# This is required for geocoding which we want to use to find addresses of deployed CCTV

# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cctv_df['Location 1']:
    # Try to,
    try:
        #Remove ( from the string
        row = row.replace("(", "")
        row = row.replace(")", "")
        # Split the row by comma, convert to float, and append
        # everything before the comma to lat
        lat.append(float(row.split(',')[0]))
        # Split the row by comma, convert to float, and append
        # everything after the comma to lon
        lon.append(float(row.split(',')[1]))
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
cctv_df['latitude'] = lat
cctv_df['longitude'] = lon

In [14]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude,neighborhood
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,NorthEutawStreet
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,RidgleysDelight
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,WestBaltimoreStreet
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,RidgleysDelight
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,RidgleysDelight


In [15]:
#dropping the columns we used to define neighborhood
cctv_df = cctv_df.drop(columns=['Location 1', 'latitude', 'longitude'])
cctv_df.head(10)

,cameraLocation,cameraNumber,cameraProject,neighborhood
0,Eutaw and Lexington Market,1,Downtown,NorthEutawStreet
1,Eutaw and Fayette,2,Downtown,RidgleysDelight
2,Eutaw and Baltimore,3,Downtown,WestBaltimoreStreet
3,Eutaw and Redwood,4,Downtown,RidgleysDelight
4,Eutaw and Lombard,5,Downtown,RidgleysDelight
5,Eutaw and Camden,6,Downtown,Baltimore
6,Paca and Pratt,7,Downtown,RidgleysDelight
7,Greene and Lombard,8,Downtown,RidgleysDelight
8,Greene and Baltimore,9,Downtown,RidgleysDelight
9,Greene and Fayette,10,Downtown,RidgleysDelight


In [16]:
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)
map_osm
